In [1]:
import os

data_path = os.getenv('HOME')+'/aiffel/conditional_generation/data/pokemon_pix2pix_dataset/train/'
print("number of train examples :", len(os.listdir(data_path)))

number of train examples : 830


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot at plt
plt.figure(figsize=(20,15))
for i in range(1, 7):
    f = data.path + os.listdir(data_path)[np.randoㅌm.randint(800)]